# VAE Training - Faces dataset

## imports

In [1]:
import os
from glob import glob
import numpy as np

from models.VAE import VariationalAutoencoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# run params
section = 'vae'
run_id = '0001'
data_name = 'faces'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


DATA_FOLDER = './data/celeb/img_align_celeba/'

## data

In [3]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)

In [4]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training"
                                            )

Found 202599 images belonging to 1 classes.


## architecture

In [5]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [6]:
vae.encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64, 64, 32)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [7]:
vae.decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 200)]             0         
_________________________________________________________________
dense (Dense)                (None, 4096)              823296    
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0   

## training

In [8]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [9]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [10]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
6331/6331 [==============================] - 217s 34ms/step - loss: 274.7881 - vae_r_loss: 211.8083 - vae_kl_loss: 62.9797

Epoch 00001: saving model to run/vae/0001_faces/weights/weights-001-274.79.h5

Epoch 00001: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 2/200
6331/6331 [==============================] - 170s 27ms/step - loss: 233.3187 - vae_r_loss: 174.5669 - vae_kl_loss: 58.7523

Epoch 00002: saving model to run/vae/0001_faces/weights/weights-002-233.32.h5

Epoch 00002: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 3/200
6331/6331 [==============================] - 172s 27ms/step - loss: 228.5017 - vae_r_loss: 169.6067 - vae_kl_loss: 58.8954

Epoch 00003: saving model to run/vae/0001_faces/weights/weights-003-228.50.h5

Epoch 00003: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 4/200
6331/6331 [==============================] - 175s 28ms/step - loss: 226.1574 - vae_r_loss: 167.2335 - vae_kl_loss: 58.9243

Epoch 00004: savin

Epoch 29/200
6331/6331 [==============================] - 168s 27ms/step - loss: 218.8068 - vae_r_loss: 159.8567 - vae_kl_loss: 58.9500

Epoch 00029: saving model to run/vae/0001_faces/weights/weights-029-218.81.h5

Epoch 00029: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 30/200
6331/6331 [==============================] - 168s 27ms/step - loss: 218.7667 - vae_r_loss: 159.8058 - vae_kl_loss: 58.9609

Epoch 00030: saving model to run/vae/0001_faces/weights/weights-030-218.77.h5

Epoch 00030: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 31/200
6331/6331 [==============================] - 168s 27ms/step - loss: 218.6537 - vae_r_loss: 159.6778 - vae_kl_loss: 58.9755

Epoch 00031: saving model to run/vae/0001_faces/weights/weights-031-218.65.h5

Epoch 00031: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 32/200
6331/6331 [==============================] - 168s 27ms/step - loss: 218.6516 - vae_r_loss: 159.7057 - vae_kl_loss: 58.9455

Epoch 00032: s

Epoch 58/200
6331/6331 [==============================] - 169s 27ms/step - loss: 217.8837 - vae_r_loss: 158.8620 - vae_kl_loss: 59.0215

Epoch 00058: saving model to run/vae/0001_faces/weights/weights-058-217.88.h5

Epoch 00058: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 59/200
6331/6331 [==============================] - 169s 27ms/step - loss: 217.8042 - vae_r_loss: 158.7876 - vae_kl_loss: 59.0172

Epoch 00059: saving model to run/vae/0001_faces/weights/weights-059-217.80.h5

Epoch 00059: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 60/200
6331/6331 [==============================] - 169s 27ms/step - loss: 217.7707 - vae_r_loss: 158.8029 - vae_kl_loss: 58.9680

Epoch 00060: saving model to run/vae/0001_faces/weights/weights-060-217.77.h5

Epoch 00060: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 61/200
6331/6331 [==============================] - 169s 27ms/step - loss: 217.7897 - vae_r_loss: 158.7822 - vae_kl_loss: 59.0072

Epoch 00061: s

Epoch 87/200
6331/6331 [==============================] - 169s 27ms/step - loss: 217.3695 - vae_r_loss: 158.3940 - vae_kl_loss: 58.9748

Epoch 00087: saving model to run/vae/0001_faces/weights/weights-087-217.37.h5

Epoch 00087: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 88/200
6331/6331 [==============================] - 170s 27ms/step - loss: 217.4360 - vae_r_loss: 158.4457 - vae_kl_loss: 58.9902

Epoch 00088: saving model to run/vae/0001_faces/weights/weights-088-217.44.h5

Epoch 00088: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 89/200
6331/6331 [==============================] - 170s 27ms/step - loss: 217.4776 - vae_r_loss: 158.4497 - vae_kl_loss: 59.0277

Epoch 00089: saving model to run/vae/0001_faces/weights/weights-089-217.48.h5

Epoch 00089: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 90/200
6331/6331 [==============================] - 169s 27ms/step - loss: 217.5169 - vae_r_loss: 158.4942 - vae_kl_loss: 59.0230

Epoch 00090: s

Epoch 116/200
6331/6331 [==============================] - 170s 27ms/step - loss: 217.2820 - vae_r_loss: 158.2791 - vae_kl_loss: 59.0029

Epoch 00116: saving model to run/vae/0001_faces/weights/weights-116-217.28.h5

Epoch 00116: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 117/200
6331/6331 [==============================] - 170s 27ms/step - loss: 217.2061 - vae_r_loss: 158.2070 - vae_kl_loss: 58.9993

Epoch 00117: saving model to run/vae/0001_faces/weights/weights-117-217.21.h5

Epoch 00117: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 118/200
6331/6331 [==============================] - 170s 27ms/step - loss: 217.2655 - vae_r_loss: 158.2659 - vae_kl_loss: 58.9998

Epoch 00118: saving model to run/vae/0001_faces/weights/weights-118-217.27.h5

Epoch 00118: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 119/200
6331/6331 [==============================] - 171s 27ms/step - loss: 217.1447 - vae_r_loss: 158.1345 - vae_kl_loss: 59.0105

Epoch 0011

Epoch 145/200
6331/6331 [==============================] - 171s 27ms/step - loss: 217.0440 - vae_r_loss: 158.0312 - vae_kl_loss: 59.0123

Epoch 00145: saving model to run/vae/0001_faces/weights/weights-145-217.04.h5

Epoch 00145: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 146/200
6331/6331 [==============================] - 170s 27ms/step - loss: 217.0361 - vae_r_loss: 158.0357 - vae_kl_loss: 58.9999

Epoch 00146: saving model to run/vae/0001_faces/weights/weights-146-217.04.h5

Epoch 00146: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 147/200
6331/6331 [==============================] - 170s 27ms/step - loss: 217.0859 - vae_r_loss: 158.0488 - vae_kl_loss: 59.0370

Epoch 00147: saving model to run/vae/0001_faces/weights/weights-147-217.09.h5

Epoch 00147: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 148/200
6331/6331 [==============================] - 171s 27ms/step - loss: 217.0901 - vae_r_loss: 158.0828 - vae_kl_loss: 59.0075

Epoch 0014

Epoch 174/200
6331/6331 [==============================] - 170s 27ms/step - loss: 216.9469 - vae_r_loss: 157.9216 - vae_kl_loss: 59.0255

Epoch 00174: saving model to run/vae/0001_faces/weights/weights-174-216.95.h5

Epoch 00174: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 175/200
6331/6331 [==============================] - 171s 27ms/step - loss: 216.9149 - vae_r_loss: 157.8867 - vae_kl_loss: 59.0283

Epoch 00175: saving model to run/vae/0001_faces/weights/weights-175-216.91.h5

Epoch 00175: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 176/200
6331/6331 [==============================] - 171s 27ms/step - loss: 216.9843 - vae_r_loss: 157.9494 - vae_kl_loss: 59.0345

Epoch 00176: saving model to run/vae/0001_faces/weights/weights-176-216.98.h5

Epoch 00176: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 177/200
6331/6331 [==============================] - 171s 27ms/step - loss: 216.9157 - vae_r_loss: 157.8734 - vae_kl_loss: 59.0428

Epoch 0017